In [1]:
import pyrouge

In [2]:
import rouge

In [3]:
#cand_file = '/home/nghia/data/git/PreSumm/logs/ext_abs_bert_wcep.1.candidate'
#gold_file = '/home/nghia/data/git/PreSumm/logs/ext_abs_bert_wcep.1.gold'

In [26]:
def get_lines(input_file):
    with open(input_file) as in_f:
        lines = [line.strip() for line in in_f]
    return lines

def compute_pyrouge(cand_file, gold_file):
    predictions = get_lines(cand_file)
    truths = get_lines(gold_file)
    good_truths = [truth for truth,cand in zip(truths, predictions) if cand != '']
    good_cands = [cand for cand in predictions if cand != '']
    scores = rouge.Rouge().get_scores(good_cands, good_truths, avg=True)
    return scores

{'rouge-1': {'f': 0.2696369283118521, 'p': 0.22951387582929023, 'r': 0.362054461924158}, 'rouge-2': {'f': 0.07258391254257883, 'p': 0.06123277088315756, 'r': 0.10219346653493633}, 'rouge-l': {'f': 0.1833401032624282, 'p': 0.17542378955708535, 'r': 0.2758674171747445}}


In [11]:
from __future__ import print_function, unicode_literals, division

import os
import re
import codecs
import platform

from subprocess import check_output
from tempfile import mkdtemp
from functools import partial

try:
    from configparser import ConfigParser
except ImportError:
    from ConfigParser import ConfigParser

from pyrouge.utils import log
from pyrouge.utils.file_utils import verify_dir


REMAP = {"-lrb-": "(", "-rrb-": ")", "-lcb-": "{", "-rcb-": "}",
         "-lsb-": "[", "-rsb-": "]", "``": '"', "''": '"'}


def clean(x):
    return re.sub(
            r"-lrb-|-rrb-|-lcb-|-rcb-|-lsb-|-rsb-|``|''",
            lambda m: REMAP.get(m.group()), x)


class DirectoryProcessor:

    @staticmethod
    def process(input_dir, output_dir, function):
        """
        Apply function to all files in input_dir and save the resulting ouput
        files in output_dir.

        """
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        logger = log.get_global_console_logger()
        logger.info("Processing files in {}.".format(input_dir))
        input_file_names = os.listdir(input_dir)
        for input_file_name in input_file_names:
            input_file = os.path.join(input_dir, input_file_name)
            with codecs.open(input_file, "r", encoding="UTF-8") as f:
                input_string = f.read()
            output_string = function(input_string)
            output_file = os.path.join(output_dir, input_file_name)
            with codecs.open(output_file, "w", encoding="UTF-8") as f:
                f.write(clean(output_string.lower()))
        logger.info("Saved processed files to {}.".format(output_dir))


class Rouge155(object):
    """
    This is a wrapper for the ROUGE 1.5.5 summary evaluation package.
    This class is designed to simplify the evaluation process by:

        1) Converting summaries into a format ROUGE understands.
        2) Generating the ROUGE configuration file automatically based
            on filename patterns.

    This class can be used within Python like this:

    rouge = Rouge155()
    rouge.system_dir = 'test/systems'
    rouge.model_dir = 'test/models'

    # The system filename pattern should contain one group that
    # matches the document ID.
    rouge.system_filename_pattern = 'SL.P.10.R.11.SL062003-(\d+).html'

    # The model filename pattern has '#ID#' as a placeholder for the
    # document ID. If there are multiple model summaries, pyrouge
    # will use the provided regex to automatically match them with
    # the corresponding system summary. Here, [A-Z] matches
    # multiple model summaries for a given #ID#.
    rouge.model_filename_pattern = 'SL.P.10.R.[A-Z].SL062003-#ID#.html'

    rouge_output = rouge.evaluate()
    print(rouge_output)
    output_dict = rouge.output_to_dict(rouge_ouput)
    print(output_dict)
    ->    {'rouge_1_f_score': 0.95652,
         'rouge_1_f_score_cb': 0.95652,
         'rouge_1_f_score_ce': 0.95652,
         'rouge_1_precision': 0.95652,
        [...]


    To evaluate multiple systems:

        rouge = Rouge155()
        rouge.system_dir = '/PATH/TO/systems'
        rouge.model_dir = 'PATH/TO/models'
        for system_id in ['id1', 'id2', 'id3']:
            rouge.system_filename_pattern = \
                'SL.P/.10.R.{}.SL062003-(\d+).html'.format(system_id)
            rouge.model_filename_pattern = \
                'SL.P.10.R.[A-Z].SL062003-#ID#.html'
            rouge_output = rouge.evaluate(system_id)
            print(rouge_output)

    """

    def __init__(self, rouge_dir=None, rouge_args=None, temp_dir = None):
        """
        Create a Rouge155 object.

            rouge_dir:  Directory containing Rouge-1.5.5.pl
            rouge_args: Arguments to pass through to ROUGE if you
                        don't want to use the default pyrouge
                        arguments.

        """
        self.temp_dir=temp_dir
        self.log = log.get_global_console_logger()
        self.__set_dir_properties()
        self._config_file = None
        self._settings_file = self.__get_config_path()
        self.__set_rouge_dir(rouge_dir)
        self.args = self.__clean_rouge_args(rouge_args)
        self._system_filename_pattern = None
        self._model_filename_pattern = None

    def save_home_dir(self):
        config = ConfigParser()
        section = 'pyrouge settings'
        config.add_section(section)
        config.set(section, 'home_dir', self._home_dir)
        with open(self._settings_file, 'w') as f:
            config.write(f)
        self.log.info("Set ROUGE home directory to {}.".format(self._home_dir))

    @property
    def settings_file(self):
        """
        Path of the setttings file, which stores the ROUGE home dir.

        """
        return self._settings_file

    @property
    def bin_path(self):
        """
        The full path of the ROUGE binary (although it's technically
        a script), i.e. rouge_home_dir/ROUGE-1.5.5.pl

        """
        if self._bin_path is None:
            raise Exception(
                "ROUGE path not set. Please set the ROUGE home directory "
                "and ensure that ROUGE-1.5.5.pl exists in it.")
        return self._bin_path

    @property
    def system_filename_pattern(self):
        """
        The regular expression pattern for matching system summary
        filenames. The regex string.

        E.g. "SL.P.10.R.11.SL062003-(\d+).html" will match the system
        filenames in the SPL2003/system folder of the ROUGE SPL example
        in the "sample-test" folder.

        Currently, there is no support for multiple systems.

        """
        return self._system_filename_pattern

    @system_filename_pattern.setter
    def system_filename_pattern(self, pattern):
        self._system_filename_pattern = pattern

    @property
    def model_filename_pattern(self):
        """
        The regular expression pattern for matching model summary
        filenames. The pattern needs to contain the string "#ID#",
        which is a placeholder for the document ID.

        E.g. "SL.P.10.R.[A-Z].SL062003-#ID#.html" will match the model
        filenames in the SPL2003/system folder of the ROUGE SPL
        example in the "sample-test" folder.

        "#ID#" is a placeholder for the document ID which has been
        matched by the "(\d+)" part of the system filename pattern.
        The different model summaries for a given document ID are
        matched by the "[A-Z]" part.

        """
        return self._model_filename_pattern

    @model_filename_pattern.setter
    def model_filename_pattern(self, pattern):
        self._model_filename_pattern = pattern

    @property
    def config_file(self):
        return self._config_file

    @config_file.setter
    def config_file(self, path):
        config_dir, _ = os.path.split(path)
        verify_dir(config_dir, "configuration file")
        self._config_file = path

    def split_sentences(self):
        """
        ROUGE requires texts split into sentences. In case the texts
        are not already split, this method can be used.

        """
        from pyrouge.utils.sentence_splitter import PunktSentenceSplitter
        self.log.info("Splitting sentences.")
        ss = PunktSentenceSplitter()
        sent_split_to_string = lambda s: "\n".join(ss.split(s))
        process_func = partial(
            DirectoryProcessor.process, function=sent_split_to_string)
        self.__process_summaries(process_func)

    @staticmethod
    def convert_summaries_to_rouge_format(input_dir, output_dir):
        """
        Convert all files in input_dir into a format ROUGE understands
        and saves the files to output_dir. The input files are assumed
        to be plain text with one sentence per line.

            input_dir:  Path of directory containing the input files.
            output_dir: Path of directory in which the converted files
                        will be saved.

        """
        DirectoryProcessor.process(
            input_dir, output_dir, Rouge155.convert_text_to_rouge_format)

    @staticmethod
    def convert_text_to_rouge_format(text, title="dummy title"):
        """
        Convert a text to a format ROUGE understands. The text is
        assumed to contain one sentence per line.

            text:   The text to convert, containg one sentence per line.
            title:  Optional title for the text. The title will appear
                    in the converted file, but doesn't seem to have
                    any other relevance.

        Returns: The converted text as string.

        """
        # sentences = text.split("\n")
        sentences = text.split("<q>")
        sent_elems = [
            "<a name=\"{i}\">[{i}]</a> <a href=\"#{i}\" id={i}>"
            "{text}</a>".format(i=i, text=sent)
            for i, sent in enumerate(sentences, start=1)]
        html = """<html>
<head>
<title>{title}</title>
</head>
<body bgcolor="white">
{elems}
</body>
</html>""".format(title=title, elems="\n".join(sent_elems))

        return html

    @staticmethod
    def write_config_static(system_dir, system_filename_pattern,
                            model_dir, model_filename_pattern,
                            config_file_path, system_id=None):
        """
        Write the ROUGE configuration file, which is basically a list
        of system summary files and their corresponding model summary
        files.

        pyrouge uses regular expressions to automatically find the
        matching model summary files for a given system summary file
        (cf. docstrings for system_filename_pattern and
        model_filename_pattern).

            system_dir:                 Path of directory containing
                                        system summaries.
            system_filename_pattern:    Regex string for matching
                                        system summary filenames.
            model_dir:                  Path of directory containing
                                        model summaries.
            model_filename_pattern:     Regex string for matching model
                                        summary filenames.
            config_file_path:           Path of the configuration file.
            system_id:                  Optional system ID string which
                                        will appear in the ROUGE output.

        """
        system_filenames = [f for f in os.listdir(system_dir)]
        system_models_tuples = []

        system_filename_pattern = re.compile(system_filename_pattern)
        for system_filename in sorted(system_filenames):
            match = system_filename_pattern.match(system_filename)
            if match:
                id = match.groups(0)[0]
                model_filenames = [model_filename_pattern.replace('#ID#',id)]
                # model_filenames = Rouge155.__get_model_filenames_for_id(
                #     id, model_dir, model_filename_pattern)
                system_models_tuples.append(
                    (system_filename, sorted(model_filenames)))
        if not system_models_tuples:
            raise Exception(
                "Did not find any files matching the pattern {} "
                "in the system summaries directory {}.".format(
                    system_filename_pattern.pattern, system_dir))

        with codecs.open(config_file_path, 'w', encoding='utf-8') as f:
            f.write('<ROUGE-EVAL version="1.55">')
            for task_id, (system_filename, model_filenames) in enumerate(
                    system_models_tuples, start=1):

                eval_string = Rouge155.__get_eval_string(
                    task_id, system_id,
                    system_dir, system_filename,
                    model_dir, model_filenames)
                f.write(eval_string)
            f.write("</ROUGE-EVAL>")

    def write_config(self, config_file_path=None, system_id=None):
        """
        Write the ROUGE configuration file, which is basically a list
        of system summary files and their matching model summary files.

        This is a non-static version of write_config_file_static().

            config_file_path:   Path of the configuration file.
            system_id:          Optional system ID string which will
                                appear in the ROUGE output.

        """
        if not system_id:
            system_id = 1
        if (not config_file_path) or (not self._config_dir):
            self._config_dir = mkdtemp(dir=self.temp_dir)
            config_filename = "rouge_conf.xml"
        else:
            config_dir, config_filename = os.path.split(config_file_path)
            verify_dir(config_dir, "configuration file")
        self._config_file = os.path.join(self._config_dir, config_filename)
        Rouge155.write_config_static(
            self._system_dir, self._system_filename_pattern,
            self._model_dir, self._model_filename_pattern,
            self._config_file, system_id)
        self.log.info(
            "Written ROUGE configuration to {}".format(self._config_file))

    def evaluate(self, system_id=1, rouge_args=None):
        """
        Run ROUGE to evaluate the system summaries in system_dir against
        the model summaries in model_dir. The summaries are assumed to
        be in the one-sentence-per-line HTML format ROUGE understands.

            system_id:  Optional system ID which will be printed in
                        ROUGE's output.

        Returns: Rouge output as string.

        """
        self.write_config(system_id=system_id)
        options = self.__get_options(rouge_args)
        command = [self._bin_path] + options
        self.log.info(
            "Running ROUGE with command {}".format(" ".join(command)))
        rouge_output = check_output(command).decode("UTF-8")
        return rouge_output

    def convert_and_evaluate(self, system_id=1,
                             split_sentences=False, rouge_args=None):
        """
        Convert plain text summaries to ROUGE format and run ROUGE to
        evaluate the system summaries in system_dir against the model
        summaries in model_dir. Optionally split texts into sentences
        in case they aren't already.

        This is just a convenience method combining
        convert_summaries_to_rouge_format() and evaluate().

            split_sentences:    Optional argument specifying if
                                sentences should be split.
            system_id:          Optional system ID which will be printed
                                in ROUGE's output.

        Returns: ROUGE output as string.

        """
        if split_sentences:
            self.split_sentences()
        self.__write_summaries()
        rouge_output = self.evaluate(system_id, rouge_args)
        return rouge_output

    def output_to_dict(self, output):
        """
        Convert the ROUGE output into python dictionary for further
        processing.

        """
        #0 ROUGE-1 Average_R: 0.02632 (95%-conf.int. 0.02632 - 0.02632)
        pattern = re.compile(
            r"(\d+) (ROUGE-\S+) (Average_\w): (\d.\d+) "
            r"\(95%-conf.int. (\d.\d+) - (\d.\d+)\)")
        results = {}
        for line in output.split("\n"):
            match = pattern.match(line)
            if match:
                sys_id, rouge_type, measure, result, conf_begin, conf_end = \
                    match.groups()
                measure = {
                    'Average_R': 'recall',
                    'Average_P': 'precision',
                    'Average_F': 'f_score'
                    }[measure]
                rouge_type = rouge_type.lower().replace("-", '_')
                key = "{}_{}".format(rouge_type, measure)
                results[key] = float(result)
                results["{}_cb".format(key)] = float(conf_begin)
                results["{}_ce".format(key)] = float(conf_end)
        return results

    ###################################################################
    # Private methods

    def __set_rouge_dir(self, home_dir=None):
        """
        Verfify presence of ROUGE-1.5.5.pl and data folder, and set
        those paths.

        """
        if not home_dir:
            self._home_dir = self.__get_rouge_home_dir_from_settings()
        else:
            self._home_dir = home_dir
            self.save_home_dir()
        self._bin_path = os.path.join(self._home_dir, 'ROUGE-1.5.5.pl')
        self.data_dir = os.path.join(self._home_dir, 'data')
        if not os.path.exists(self._bin_path):
            raise Exception(
                "ROUGE binary not found at {}. Please set the "
                "correct path by running pyrouge_set_rouge_path "
                "/path/to/rouge/home.".format(self._bin_path))

    def __get_rouge_home_dir_from_settings(self):
        config = ConfigParser()
        with open(self._settings_file) as f:
            if hasattr(config, "read_file"):
                config.read_file(f)
            else:
                # use deprecated python 2.x method
                config.readfp(f)
        rouge_home_dir = config.get('pyrouge settings', 'home_dir')
        return rouge_home_dir

    @staticmethod
    def __get_eval_string(
            task_id, system_id,
            system_dir, system_filename,
            model_dir, model_filenames):
        """
        ROUGE can evaluate several system summaries for a given text
        against several model summaries, i.e. there is an m-to-n
        relation between system and model summaries. The system
        summaries are listed in the <PEERS> tag and the model summaries
        in the <MODELS> tag. pyrouge currently only supports one system
        summary per text, i.e. it assumes a 1-to-n relation between
        system and model summaries.

        """
        peer_elems = "<P ID=\"{id}\">{name}</P>".format(
            id=system_id, name=system_filename)

        model_elems = ["<M ID=\"{id}\">{name}</M>".format(
            id=chr(65 + i), name=name)
            for i, name in enumerate(model_filenames)]

        model_elems = "\n\t\t\t".join(model_elems)
        eval_string = """
    <EVAL ID="{task_id}">
        <MODEL-ROOT>{model_root}</MODEL-ROOT>
        <PEER-ROOT>{peer_root}</PEER-ROOT>
        <INPUT-FORMAT TYPE="SEE">
        </INPUT-FORMAT>
        <PEERS>
            {peer_elems}
        </PEERS>
        <MODELS>
            {model_elems}
        </MODELS>
    </EVAL>
""".format(
            task_id=task_id,
            model_root=model_dir, model_elems=model_elems,
            peer_root=system_dir, peer_elems=peer_elems)
        return eval_string

    def __process_summaries(self, process_func):
        """
        Helper method that applies process_func to the files in the
        system and model folders and saves the resulting files to new
        system and model folders.

        """
        temp_dir = mkdtemp(dir=self.temp_dir)
        new_system_dir = os.path.join(temp_dir, "system")
        os.mkdir(new_system_dir)
        new_model_dir = os.path.join(temp_dir, "model")
        os.mkdir(new_model_dir)
        self.log.info(
            "Processing summaries. Saving system files to {} and "
            "model files to {}.".format(new_system_dir, new_model_dir))
        process_func(self._system_dir, new_system_dir)
        process_func(self._model_dir, new_model_dir)
        self._system_dir = new_system_dir
        self._model_dir = new_model_dir

    def __write_summaries(self):
        self.log.info("Writing summaries.")
        self.__process_summaries(self.convert_summaries_to_rouge_format)

    @staticmethod
    def __get_model_filenames_for_id(id, model_dir, model_filenames_pattern):
        pattern = re.compile(model_filenames_pattern.replace('#ID#', id))
        model_filenames = [
            f for f in os.listdir(model_dir) if pattern.match(f)]
        if not model_filenames:
            raise Exception(
                "Could not find any model summaries for the system"
                " summary with ID {}. Specified model filename pattern was: "
                "{}".format(id, model_filenames_pattern))
        return model_filenames

    def __get_options(self, rouge_args=None):
        """
        Get supplied command line arguments for ROUGE or use default
        ones.

        """
        if self.args:
            options = self.args.split()
        elif rouge_args:
            options = rouge_args.split()
        else:
            options = [
                '-e', self._data_dir,
                '-c', 95,
                # '-2',
                # '-1',
                # '-U',
                '-m',
                # '-v',
                '-r', 1000,
                '-n', 2,
                # '-w', 1.2,
                '-a',
                ]
            options = list(map(str, options))




        options = self.__add_config_option(options)
        return options

    def __create_dir_property(self, dir_name, docstring):
        """
        Generate getter and setter for a directory property.

        """
        property_name = "{}_dir".format(dir_name)
        private_name = "_" + property_name
        setattr(self, private_name, None)

        def fget(self):
            return getattr(self, private_name)

        def fset(self, path):
            verify_dir(path, dir_name)
            setattr(self, private_name, path)

        p = property(fget=fget, fset=fset, doc=docstring)
        setattr(self.__class__, property_name, p)

    def __set_dir_properties(self):
        """
        Automatically generate the properties for directories.

        """
        directories = [
            ("home", "The ROUGE home directory."),
            ("data", "The path of the ROUGE 'data' directory."),
            ("system", "Path of the directory containing system summaries."),
            ("model", "Path of the directory containing model summaries."),
            ]
        for (dirname, docstring) in directories:
            self.__create_dir_property(dirname, docstring)

    def __clean_rouge_args(self, rouge_args):
        """
        Remove enclosing quotation marks, if any.

        """
        if not rouge_args:
            return
        quot_mark_pattern = re.compile('"(.+)"')
        match = quot_mark_pattern.match(rouge_args)
        if match:
            cleaned_args = match.group(1)
            return cleaned_args
        else:
            return rouge_args

    def __add_config_option(self, options):
        return options + [self._config_file]

    def __get_config_path(self):
        if platform.system() == "Windows":
            parent_dir = os.getenv("APPDATA")
            config_dir_name = "pyrouge"
        elif os.name == "posix":
            parent_dir = os.path.expanduser("~")
            config_dir_name = ".pyrouge"
        else:
            parent_dir = os.path.dirname(__file__)
            config_dir_name = ""
        config_dir = os.path.join(parent_dir, config_dir_name)
        if not os.path.exists(config_dir):
            os.makedirs(config_dir)
        return os.path.join(config_dir, 'settings.ini')

In [15]:
import os
import re
import shutil
import time
def test_rouge(temp_dir, cand, ref):
    candidates = [line.strip() for line in open(cand, encoding='utf-8')]
    references = [line.strip() for line in open(ref, encoding='utf-8')]
    print(len(candidates))
    print(len(references))
    assert len(candidates) == len(references)
    
    #scores = rouge.Rouge().get_scores(candidates, references, avg=True)
    #return scores
    
    cnt = len(candidates)
    current_time = time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime())
    tmp_dir = os.path.join(temp_dir, "rouge-tmp-{}".format(current_time))
    if not os.path.isdir(tmp_dir):
        os.mkdir(tmp_dir)
        os.mkdir(tmp_dir + "/candidate")
        os.mkdir(tmp_dir + "/reference")
    try:

        for i in range(cnt):
            if len(references[i]) < 1:
                continue
            with open(tmp_dir + "/candidate/cand.{}.txt".format(i), "w",
                      encoding="utf-8") as f:
                f.write(candidates[i])
            with open(tmp_dir + "/reference/ref.{}.txt".format(i), "w",
                      encoding="utf-8") as f:
                f.write(references[i])
        r = Rouge155(temp_dir=temp_dir)
        r.model_dir = tmp_dir + "/reference/"
        r.system_dir = tmp_dir + "/candidate/"
        r.model_filename_pattern = 'ref.#ID#.txt'
        r.system_filename_pattern = r'cand.(\d+).txt'
        rouge_results = r.convert_and_evaluate()
        print(rouge_results)
        results_dict = r.output_to_dict(rouge_results)
    finally:
        pass
        if os.path.isdir(tmp_dir):
            shutil.rmtree(tmp_dir)
    return results_dict

In [16]:
temp_dir = '/home/nghia/data/git/PreSumm/logs/temp/'

In [61]:
cand_file = '/home/nghia/data/datasets/summarization/cnndm/textrank/cnndm.test.0.cand'
gold_file = '/home/nghia/data/datasets/summarization/cnndm/textrank/cnndm.test.0.gold'

In [62]:
def shortern_summary(summary):
    sentences = summary.split(' <q> ')
    new_length = min(len(sentences), 2)
    return '<q>'.join(sentences[0:new_length])

def print_lines(lines, output_file):
    with open(output_file, 'w') as out_f:
        for line in lines:
            out_f.write(f'{line}\n')

In [63]:
short_cand_file = '/'.join(cand_file.split('/')[0:-1]) + '/short.' + cand_file.split('/')[-1]
print(short_cand_file)
lines = get_lines(cand_file)
short_lines = [shortern_summary(line) for line in lines]
print_lines(short_lines, short_cand_file)

/home/nghia/data/datasets/summarization/cnndm/textrank/short.cnndm.test.0.cand


In [64]:
#test_rouge(temp_dir, cand_file, gold_file)
#print(compute_pyrouge(cand_file, gold_file))
test_rouge(temp_dir, short_cand_file, gold_file)
print(compute_pyrouge(short_cand_file, gold_file))

2019-12-11 10:22:36,689 [MainThread  ] [INFO ]  Writing summaries.
2019-12-11 10:22:36,690 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /home/nghia/data/git/PreSumm/logs/temp/tmp84aatsm8/system and model files to /home/nghia/data/git/PreSumm/logs/temp/tmp84aatsm8/model.
2019-12-11 10:22:36,690 [MainThread  ] [INFO ]  Processing files in /home/nghia/data/git/PreSumm/logs/temp/rouge-tmp-2019-12-11-10-22-36/candidate/.


1801
1801


2019-12-11 10:22:36,794 [MainThread  ] [INFO ]  Saved processed files to /home/nghia/data/git/PreSumm/logs/temp/tmp84aatsm8/system.
2019-12-11 10:22:36,794 [MainThread  ] [INFO ]  Processing files in /home/nghia/data/git/PreSumm/logs/temp/rouge-tmp-2019-12-11-10-22-36/reference/.
2019-12-11 10:22:36,909 [MainThread  ] [INFO ]  Saved processed files to /home/nghia/data/git/PreSumm/logs/temp/tmp84aatsm8/model.
2019-12-11 10:22:36,919 [MainThread  ] [INFO ]  Written ROUGE configuration to /home/nghia/data/git/PreSumm/logs/temp/tmpslpx1hew/rouge_conf.xml
2019-12-11 10:22:36,920 [MainThread  ] [INFO ]  Running ROUGE with command /home/nghia/data/git/pyrouge_tools/tools/ROUGE-1.5.5/ROUGE-1.5.5.pl -e /home/nghia/data/git/pyrouge_tools/tools/ROUGE-1.5.5/data -c 95 -m -r 1000 -n 2 -a /home/nghia/data/git/PreSumm/logs/temp/tmpslpx1hew/rouge_conf.xml


---------------------------------------------
1 ROUGE-1 Average_R: 0.36408 (95%-conf.int. 0.35747 - 0.37095)
1 ROUGE-1 Average_P: 0.39167 (95%-conf.int. 0.38432 - 0.39908)
1 ROUGE-1 Average_F: 0.35708 (95%-conf.int. 0.35140 - 0.36293)
---------------------------------------------
1 ROUGE-2 Average_R: 0.14965 (95%-conf.int. 0.14385 - 0.15551)
1 ROUGE-2 Average_P: 0.16333 (95%-conf.int. 0.15654 - 0.16990)
1 ROUGE-2 Average_F: 0.14745 (95%-conf.int. 0.14184 - 0.15334)
---------------------------------------------
1 ROUGE-L Average_R: 0.32591 (95%-conf.int. 0.31967 - 0.33260)
1 ROUGE-L Average_P: 0.35161 (95%-conf.int. 0.34450 - 0.35901)
1 ROUGE-L Average_F: 0.32004 (95%-conf.int. 0.31437 - 0.32570)

{'rouge-1': {'f': 0.34877930090040943, 'p': 0.38163056095684045, 'r': 0.34674919638225393}, 'rouge-2': {'f': 0.1347819228276147, 'p': 0.15331478539589186, 'r': 0.13358619740840852}, 'rouge-l': {'f': 0.25172190236000747, 'p': 0.2913034017214763, 'r': 0.26705682094696165}}


In [38]:
cand_file = '/home/nghia/data/git/PreSumm/logs/ext_bert_cnndm_step1.candidate'
gold_file = '/home/nghia/data/git/PreSumm/logs/ext_bert_cnndm_step1.gold'
test_rouge(temp_dir, cand_file, gold_file)
print(compute_pyrouge(cand_file, gold_file))

11489
11489


2019-12-10 17:54:09,089 [MainThread  ] [INFO ]  Writing summaries.
2019-12-10 17:54:09,090 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /home/nghia/data/git/PreSumm/logs/temp/tmpvhf66p5g/system and model files to /home/nghia/data/git/PreSumm/logs/temp/tmpvhf66p5g/model.
2019-12-10 17:54:09,090 [MainThread  ] [INFO ]  Processing files in /home/nghia/data/git/PreSumm/logs/temp/rouge-tmp-2019-12-10-17-54-08/candidate/.
2019-12-10 17:54:09,730 [MainThread  ] [INFO ]  Saved processed files to /home/nghia/data/git/PreSumm/logs/temp/tmpvhf66p5g/system.
2019-12-10 17:54:09,731 [MainThread  ] [INFO ]  Processing files in /home/nghia/data/git/PreSumm/logs/temp/rouge-tmp-2019-12-10-17-54-08/reference/.
2019-12-10 17:54:10,358 [MainThread  ] [INFO ]  Saved processed files to /home/nghia/data/git/PreSumm/logs/temp/tmpvhf66p5g/model.
2019-12-10 17:54:10,426 [MainThread  ] [INFO ]  Written ROUGE configuration to /home/nghia/data/git/PreSumm/logs/temp/tmp698kstot/rouge_conf.xml

---------------------------------------------
1 ROUGE-1 Average_R: 0.53377 (95%-conf.int. 0.53109 - 0.53650)
1 ROUGE-1 Average_P: 0.38181 (95%-conf.int. 0.37940 - 0.38435)
1 ROUGE-1 Average_F: 0.43053 (95%-conf.int. 0.42843 - 0.43265)
---------------------------------------------
1 ROUGE-2 Average_R: 0.24965 (95%-conf.int. 0.24688 - 0.25243)
1 ROUGE-2 Average_P: 0.17934 (95%-conf.int. 0.17716 - 0.18164)
1 ROUGE-2 Average_F: 0.20159 (95%-conf.int. 0.19929 - 0.20386)
---------------------------------------------
1 ROUGE-L Average_R: 0.48854 (95%-conf.int. 0.48591 - 0.49132)
1 ROUGE-L Average_P: 0.35008 (95%-conf.int. 0.34772 - 0.35261)
1 ROUGE-L Average_F: 0.39447 (95%-conf.int. 0.39236 - 0.39665)

{'rouge-1': {'f': 0.39669260370464027, 'p': 0.35191032705297043, 'r': 0.4834931556021214}, 'rouge-2': {'f': 0.17227903922901988, 'p': 0.15004092600188088, 'r': 0.22036279912569426}, 'rouge-l': {'f': 0.2920544357027657, 'p': 0.2787945871966778, 'r': 0.3851068003189376}}


In [57]:
def split_line(line):
    return ' <q> '.join(line.split('<q>'))

def split_q(input_file):
    split_file = '/'.join(input_file.split('/')[0:-1]) + '/split.' + input_file.split('/')[-1]
    lines = get_lines(input_file)
    split_lines = [split_line(line) for line in lines]
    print_lines(split_lines, split_file)
    return split_file

In [58]:
cand_file = '/home/nghia/data/git/PreSumm/logs/ext_bert_cnndm_step1.candidate'
gold_file = '/home/nghia/data/git/PreSumm/logs/ext_bert_cnndm_step1.gold'
split_cand_file = split_q(cand_file)
split_gold_file = split_q(gold_file)
test_rouge(temp_dir, split_cand_file, split_gold_file)
print(compute_pyrouge(split_cand_file, split_gold_file))

11489
11489


2019-12-11 10:07:56,336 [MainThread  ] [INFO ]  Writing summaries.
2019-12-11 10:07:56,337 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /home/nghia/data/git/PreSumm/logs/temp/tmpdy6r7kw9/system and model files to /home/nghia/data/git/PreSumm/logs/temp/tmpdy6r7kw9/model.
2019-12-11 10:07:56,337 [MainThread  ] [INFO ]  Processing files in /home/nghia/data/git/PreSumm/logs/temp/rouge-tmp-2019-12-11-10-07-55/candidate/.
2019-12-11 10:07:56,974 [MainThread  ] [INFO ]  Saved processed files to /home/nghia/data/git/PreSumm/logs/temp/tmpdy6r7kw9/system.
2019-12-11 10:07:56,976 [MainThread  ] [INFO ]  Processing files in /home/nghia/data/git/PreSumm/logs/temp/rouge-tmp-2019-12-11-10-07-55/reference/.
2019-12-11 10:07:57,615 [MainThread  ] [INFO ]  Saved processed files to /home/nghia/data/git/PreSumm/logs/temp/tmpdy6r7kw9/model.
2019-12-11 10:07:57,680 [MainThread  ] [INFO ]  Written ROUGE configuration to /home/nghia/data/git/PreSumm/logs/temp/tmpp55j45xi/rouge_conf.xml

---------------------------------------------
1 ROUGE-1 Average_R: 0.53377 (95%-conf.int. 0.53109 - 0.53650)
1 ROUGE-1 Average_P: 0.38181 (95%-conf.int. 0.37940 - 0.38435)
1 ROUGE-1 Average_F: 0.43053 (95%-conf.int. 0.42843 - 0.43265)
---------------------------------------------
1 ROUGE-2 Average_R: 0.24965 (95%-conf.int. 0.24688 - 0.25243)
1 ROUGE-2 Average_P: 0.17934 (95%-conf.int. 0.17716 - 0.18164)
1 ROUGE-2 Average_F: 0.20159 (95%-conf.int. 0.19929 - 0.20386)
---------------------------------------------
1 ROUGE-L Average_R: 0.48854 (95%-conf.int. 0.48591 - 0.49132)
1 ROUGE-L Average_P: 0.35008 (95%-conf.int. 0.34772 - 0.35261)
1 ROUGE-L Average_F: 0.39447 (95%-conf.int. 0.39236 - 0.39665)

{'rouge-1': {'f': 0.4486077569817909, 'p': 0.40524537948233824, 'r': 0.5311803688144133}, 'rouge-2': {'f': 0.19909952266570707, 'p': 0.17817967095711168, 'r': 0.24412403342428174}, 'rouge-l': {'f': 0.3298596293554962, 'p': 0.31726992531692333, 'r': 0.4175155116312289}}
